In [1]:
)clear
⎕IO ← 0
⎕CT ← 1e¯10
Assert ← {⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}   ⍝ by Roger Hui
Table  ← {⎕PP←3 ⋄ ⍕¨¨⍺,[¯0.5]⍵}                              ⍝ display as table

clear ws

# Matrix divide
The [`⌹` function](https://help.dyalog.com/latest/Content/Language/Primitive%20Functions/Matrix%20Divide.htm), when used dyadically (`b⌹m`), performs the matrix division of `b` by `m`. In case `b` is a vector and `m` is a matrix with shape `2⍴≢b`, the result of `b⌹m` will be the vector `x` which solves the linear system `b ≡ m +.× x`.

In [2]:
b m←{?⍵⍴0}¨3 (3 3)  ⍝ random sample data
'bmx' Table b m (x←b⌹m)
Assert b ≡ m +.× x

┌───────────────────┬───────────────────┬──────────────────┐
│b │m │x │
├───────────────────┼───────────────────┼──────────────────┤
│┌─────┬─────┬─────┐│┌─────┬─────┬─────┐│┌───┬──────┬─────┐│
││0.946│0.151│0.527│││0.916│0.131│0.874│││0.2│¯0.636│0.968││
│└─────┴─────┴─────┘│├─────┼─────┼─────┤│└───┴──────┴─────┘│
│ ││0.766│0.161│0.103││ │
│ │├─────┼─────┼─────┤│ │
│ ││0.907│0.563│0.727││ │
│ │└─────┴─────┴─────┘│ │
└───────────────────┴───────────────────┴──────────────────┘

# Matrix divide of nested vector elements

Sometimes, we do not need to solve a linear system, but many linear systems. What if every element of our vector and every element of our matrix was an array with several values? Let's try it. We are going to divide a vector of 2-elements vectors by a matrix of 2-elements vectors:

In [3]:
m b←2{↓?(⍵,⍺)⍴0}¨(3 3) 3  ⍝ random sample data
(,⍉'' '⍴' '⍴¨'∘.,'bm') Table b (⍴b) (⍴¨b) m (⍴m) (⍴¨m)
b⌹m

┌─────────────────────────────────────┬───┬───────┬───────────────────────────────────────┬─────┬───────┐
│b │⍴b │⍴¨b │m │⍴m │⍴¨m │
├─────────────────────────────────────┼───┼───────┼───────────────────────────────────────┼─────┼───────┤
│┌───────────┬───────────┬───────────┐│┌─┐│┌─┬─┬─┐│┌────────────┬───────────┬────────────┐│┌─┬─┐│┌─┬─┬─┐│
││0.593 0.632│0.246 0.457│0.418 0.631│││3│││2│2│2│││0.55 0.368 │0.147 0.659│0.474 0.0899│││3│3│││2│2│2││
│└───────────┴───────────┴───────────┘│└─┘│└─┴─┴─┘│├────────────┼───────────┼────────────┤│└─┴─┘│├─┼─┼─┤│
│ │ │ ││0.782 0.0688│0.677 0.773│0.587 0.381 ││ ││2│2│2││
│ │ │ │├────────────┼───────────┼────────────┤│ │├─┼─┼─┤│
│ │ │ ││0.168 0.861 │0.936 0.816│0.585 0.0458││ ││2│2│2││
│ │ │ │└────────────┴───────────┴────────────┘│ │└─┴─┴─┘│
└─────────────────────────────────────┴───┴───────┴───────────────────────────────────────┴─────┴───────┘

DOMAIN ERROR
      b⌹m
       ∧


It does not work. We need to restructure the data so that we can use the [rank operator](https://help.dyalog.com/latest/Content/Language/Primitive%20Operators/Rank.htm), `⍤`. First, we mix the arrays, to get arrays of more dimensions:

In [4]:
(,'' '⍴'∘.,(⊂'↑[0]'),¨'bm') Table (⊢,⍴¨)↑[0]¨b m

┌───────────────────┬─────────────────────┬──────┬───────┐
│↑[0]b │↑[0]m │⍴↑[0]b│⍴↑[0]m │
├───────────────────┼─────────────────────┼──────┼───────┤
│┌─────┬─────┬─────┐│┌──────┬─────┬──────┐│┌─┬─┐ │┌─┬─┬─┐│
││0.593│0.246│0.418│││0.55 │0.147│0.474 │││2│3│ ││2│3│3││
│├─────┼─────┼─────┤│├──────┼─────┼──────┤│└─┴─┘ │└─┴─┴─┘│
││0.632│0.457│0.631│││0.782 │0.677│0.587 ││ │ │
│└─────┴─────┴─────┘│├──────┼─────┼──────┤│ │ │
│ ││0.168 │0.936│0.585 ││ │ │
│ │└──────┴─────┴──────┘│ │ │
│ │┌──────┬─────┬──────┐│ │ │
│ ││0.368 │0.659│0.0899││ │ │
│ │├──────┼─────┼──────┤│ │ │
│ ││0.0688│0.773│0.381 ││ │ │
│ │├──────┼─────┼──────┤│ │ │
│ ││0.861 │0.816│0.0458││ │ │
│ │└──────┴─────┴──────┘│ │ │
└───────────────────┴─────────────────────┴──────┴───────┘

Now, matrix divide with the rank operator will work:

In [5]:
'bmx' Table b m (x←b⌹⍤1 2⍥(↑[0])m)

┌─────────────────────────────────────┬───────────────────────────────────────┬─────────────────────┐
│b │m │x │
├─────────────────────────────────────┼───────────────────────────────────────┼─────────────────────┤
│┌───────────┬───────────┬───────────┐│┌────────────┬───────────┬────────────┐│┌──────┬──────┬─────┐│
││0.593 0.632│0.246 0.457│0.418 0.631│││0.55 0.368 │0.147 0.659│0.474 0.0899│││¯0.649│¯0.854│2.27 ││
│└───────────┴───────────┴───────────┘│├────────────┼───────────┼────────────┤│├──────┼──────┼─────┤│
│ ││0.782 0.0688│0.677 0.773│0.587 0.381 │││¯0.687│1.61 │¯1.94││
│ │├────────────┼───────────┼────────────┤│└──────┴──────┴─────┘│
│ ││0.168 0.861 │0.936 0.816│0.585 0.0458││ │
│ │└────────────┴───────────┴────────────┘│ │
└─────────────────────────────────────┴───────────────────────────────────────┴─────────────────────┘

We already solved the linear system. However, the solution does not have the right structure:

In [6]:
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x)
b ≡ m +.× x

┌─────────────────────┬─────┬────┐
│x │⍴x │⍴¨x │
├─────────────────────┼─────┼────┤
│┌──────┬──────┬─────┐│┌─┬─┐│┌┬┬┐│
││¯0.649│¯0.854│2.27 │││2│3│││││││
│├──────┼──────┼─────┤│└─┴─┘│├┼┼┤│
││¯0.687│1.61 │¯1.94││ ││││││
│└──────┴──────┴─────┘│ │└┴┴┘│
└─────────────────────┴─────┴────┘

LENGTH ERROR
      b≡m+.×x
          ∧


We need to restructure the array again, so we split it, removing the first axis that we added before. This is getting a bit complicated, so we will define a *nested divide* function:

In [7]:
⎕ ← ND ← ↓[0]⌹⍤1 2⍥(↑[0])  ⍝ nested matrix divide
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
Assert b ≡ m +.× x

┌─┴──┐
 [0] ⍥
┌─┘ ┌──┴───┐
↓ ⍤ [0]
 ┌┴┐ ┌─┘
 ⌹ 1 2 ↑

┌──────────────────────────────────────┬───┬───────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────┼───┼───────┤
│┌─────────────┬───────────┬──────────┐│┌─┐│┌─┬─┬─┐│
││¯0.649 ¯0.687│¯0.854 1.61│2.27 ¯1.94│││3│││2│2│2││
│└─────────────┴───────────┴──────────┘│└─┘│└─┴─┴─┘│
└──────────────────────────────────────┴───┴───────┘

We solved the two linear systems now! This time, we got a solution array `x` with exactly the same structure as `b`, so there is no error.

# Matrix divide of nested array elements

We are not done yet. What if our nested elements are not simple vectors but arrays with rank higher than 1?

In [8]:
RN ← {5×⊂⍤(≢⍺)?(⍵,⍺)⍴0}  ⍝ random nested. ⍺ shape of element, ⍵ shape of array
m b←(⊂3 4)RN¨(2 2) 2  ⍝ random sample data
('' '⍴' '⍴¨',¨'m') Table m (⍴m) (⍴¨m)
('' '⍴' '⍴¨',¨'b') Table b (⍴b) (⍴¨b)
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
b ≡ m +.× x

┌───────────────────────────────────────────────┬─────┬─────────┐
│m │⍴m │⍴¨m │
├───────────────────────────────────────────────┼─────┼─────────┤
│┌────────────────────────┬────────────────────┐│┌─┬─┐│┌───┬───┐│
││3.06 0.907 2.03 0.701 │3.14 1.69 3.38 3.03│││2│2│││3 4│3 4││
││1.08 3.44 1.2 1.22 │0.865 4.19 4.46 4.6 ││└─┴─┘│├───┼───┤│
││0.557 0.487 2.9 2.71 │4.85 1.56 2.79 3.58││ ││3 4│3 4││
│├────────────────────────┼────────────────────┤│ │└───┴───┘│
││1.82 4.02 2.23 0.666 │1.64 3.89 3.13 2.74 ││ │ │
││4.29 0.673 1.77 0.0489│4.87 3.3 0.02 2.75 ││ │ │
││0.207 0.0942 2.35 0.0887│3.85 4.33 1.59 2.03 ││ │ │
│└────────────────────────┴────────────────────┘│ │ │
└───────────────────────────────────────────────┴─────┴─────────┘

┌───────────────────────────────────────────┬───┬─────────┐
│b │⍴b │⍴¨b │
├───────────────────────────────────────────┼───┼─────────┤
│┌─────────────────────┬───────────────────┐│┌─┐│┌───┬───┐│
││3.76 1.81 4.32 3.34 │1.71 3.71 3.76 2.83│││2│││3 4│3 4││
││0.381 4.84 3.23 0.658│2.82 2.83 4.2 3.08││└─┘│└───┴───┘│
││0.288 2.7 2.52 4.29 │1.88 2.67 2.92 3.49││ │ │
│└─────────────────────┴───────────────────┘│ │ │
└───────────────────────────────────────────┴───┴─────────┘

┌──────────────────────────────────────┬─────┬─────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────┼─────┼─────┤
│┌──────────────────┬─────────────────┐│┌─┬─┐│┌─┬─┐│
││¯1.17 ¯0.382 ¯7.04│2.34 0.916 0.867 │││4│2│││3│3││
│├──────────────────┼─────────────────┤│└─┴─┘│├─┼─┤│
││¯0.241 0.481 3.82 │1.2 0.76 0.535 ││ ││3│3││
│├──────────────────┼─────────────────┤│ │├─┼─┤│
││¯0.714 2.37 2.11 │1.71 0.0855 ¯1.29││ ││3│3││
│├──────────────────┼─────────────────┤│ │├─┼─┤│
││¯6.11 ¯3.96 ¯0.733│2.51 1.19 1.75 ││ ││3│3││
│└──────────────────┴─────────────────┘│ │└─┴─┘│
└──────────────────────────────────────┴─────┴─────┘

LENGTH ERROR
      b≡m+.×x
          ∧


We actually solved the system, but obviously `↓[0]` was not enough to restructure the result value as needed. We only removed one axis, but we added two with `↑[0]`. Since we want to nest more than one axis, we will use the [enclose function](), `⊂`, together with the rank operator, instead of split. But there is another problem. Let's have a look at the shape of the result of the matrix divide:

In [9]:
('' '⍴' '⍴¨',¨'d') Table d (⍴d) (⍴¨d←b⌹⍤1 2⍥(↑[0])m) 

┌───────────────┬───────┬───┐
│d │⍴d │⍴¨d│
├───────────────┼───────┼───┤
│┌──────┬──────┐│┌─┬─┬─┐│┌┬┐│
││¯1.17 │2.34 │││3│4│2││││││
│├──────┼──────┤│└─┴─┴─┘│├┼┤│
││¯0.241│1.2 ││ │││││
│├──────┼──────┤│ │├┼┤│
││¯0.714│1.71 ││ │││││
│├──────┼──────┤│ │├┼┤│
││¯6.11 │2.51 ││ │││││
│└──────┴──────┘│ │└┴┘│
│┌──────┬──────┐│ │┌┬┐│
││¯0.382│0.916 ││ │││││
│├──────┼──────┤│ │├┼┤│
││0.481 │0.76 ││ │││││
│├──────┼──────┤│ │├┼┤│
││2.37 │0.0855││ │││││
│├──────┼──────┤│ │├┼┤│
││¯3.96 │1.19 ││ │││││
│└──────┴──────┘│ │└┴┘│
│┌──────┬──────┐│ │┌┬┐│
││¯7.04 │0.867 ││ │││││
│├──────┼──────┤│ │├┼┤│
││3.82 │0.535 ││ │││││
│├──────┼──────┤│ │├┼┤│
││2.11 │¯1.29 ││ │││││
│├──────┼──────┤│ │├┼┤│
││¯0.733│1.75 ││ │││││
│└──────┴──────┘│ │└┴┘│
└───────────────┴───────┴───┘

What we want to do is to have the same structure of `b` (a vector of 2 elements, each of them a 3 4 array). The goal is to get rid of the first two axes, but enclose will remove the last ones. So, we need to transpose the array first. 

This might be done with a dyadic transpose. However, then we would need to first find the length of the current shape, then generate the indices, rotate them and do the transpose. Instead, we will use two monadic transposes with different ranks, which will have the same effect. First, we do a transpose reversing all the elements of the shape, then we do another transpose but this one with rank ¯1, to reverse only the shape of the subarrays, while leaving the first axis unchanged:

In [10]:
Assert (⍉⍤¯1⍉d) ≡ (1⌽⍳≢⍴d)⍉d     ⍝ two monadic transposes match dyadic transpose
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⍉d)      ⍝ first monadic transpose (rank ⍴⍴d)
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⍉⍤¯1⍉d)  ⍝ second monadic transpose (rank (⍴⍴d)-1)

┌──────────────────────┬───────┬────┐
│x │⍴x │⍴¨x │
├──────────────────────┼───────┼────┤
│┌──────┬──────┬──────┐│┌─┬─┬─┐│┌┬┬┐│
││¯1.17 │¯0.382│¯7.04 │││2│4│3│││││││
│├──────┼──────┼──────┤│└─┴─┴─┘│├┼┼┤│
││¯0.241│0.481 │3.82 ││ ││││││
│├──────┼──────┼──────┤│ │├┼┼┤│
││¯0.714│2.37 │2.11 ││ ││││││
│├──────┼──────┼──────┤│ │├┼┼┤│
││¯6.11 │¯3.96 │¯0.733││ ││││││
│└──────┴──────┴──────┘│ │└┴┴┘│
│┌──────┬──────┬──────┐│ │┌┬┬┐│
││2.34 │0.916 │0.867 ││ ││││││
│├──────┼──────┼──────┤│ │├┼┼┤│
││1.2 │0.76 │0.535 ││ ││││││
│├──────┼──────┼──────┤│ │├┼┼┤│
││1.71 │0.0855│¯1.29 ││ ││││││
│├──────┼──────┼──────┤│ │├┼┼┤│
││2.51 │1.19 │1.75 ││ ││││││
│└──────┴──────┴──────┘│ │└┴┴┘│
└──────────────────────┴───────┴────┘

┌─────────────────────────────┬───────┬─────┐
│x │⍴x │⍴¨x │
├─────────────────────────────┼───────┼─────┤
│┌──────┬──────┬──────┬──────┐│┌─┬─┬─┐│┌┬┬┬┐│
││¯1.17 │¯0.241│¯0.714│¯6.11 │││2│3│4││││││││
│├──────┼──────┼──────┼──────┤│└─┴─┴─┘│├┼┼┼┤│
││¯0.382│0.481 │2.37 │¯3.96 ││ │││││││
│├──────┼──────┼──────┼──────┤│ │├┼┼┼┤│
││¯7.04 │3.82 │2.11 │¯0.733││ │││││││
│└──────┴──────┴──────┴──────┘│ │└┴┴┴┘│
│┌──────┬──────┬──────┬──────┐│ │┌┬┬┬┐│
││2.34 │1.2 │1.71 │2.51 ││ │││││││
│├──────┼──────┼──────┼──────┤│ │├┼┼┼┤│
││0.916 │0.76 │0.0855│1.19 ││ │││││││
│├──────┼──────┼──────┼──────┤│ │├┼┼┼┤│
││0.867 │0.535 │¯1.29 │1.75 ││ │││││││
│└──────┴──────┴──────┴──────┘│ │└┴┴┴┘│
└─────────────────────────────┴───────┴─────┘

Almost there. We just need to enclose with rank two to nest the subarrays, removing the last two elements of the shape:

In [11]:
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⊂⍤¯1⍉⍤¯1⍉d)

┌──────────────────────────────────────────────────────┬───┬─────────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────────────────────┼───┼─────────┤
│┌───────────────────────────┬────────────────────────┐│┌─┐│┌───┬───┐│
││¯1.17 ¯0.241 ¯0.714 ¯6.11 │2.34 1.2 1.71 2.51│││2│││3 4│3 4││
││¯0.382 0.481 2.37 ¯3.96 │0.916 0.76 0.0855 1.19││└─┘│└───┴───┘│
││¯7.04 3.82 2.11 ¯0.733│0.867 0.535 ¯1.29 1.75││ │ │
│└───────────────────────────┴────────────────────────┘│ │ │
└──────────────────────────────────────────────────────┴───┴─────────┘

Done! Let's redefine our nested divide function:

In [12]:
⎕ ← ND ← ⊂∘⍉⍤¯1∘⍉⌹⍤1 2⍥(↑[0])  ⍝ nested matrix divide
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
Assert b ≡ m +.× x

┌──┴───┐
 ∘ ⍥
 ┌┴┐ ┌──┴───┐
 ⍤ ⍉ ⍤ [0]
 ┌┴┐ ┌┴┐ ┌─┘
 ∘ ¯1 ⌹ 1 2 ↑
┌┴┐
⊂ ⍉

┌──────────────────────────────────────────────────────┬───┬─────────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────────────────────┼───┼─────────┤
│┌───────────────────────────┬────────────────────────┐│┌─┐│┌───┬───┐│
││¯1.17 ¯0.241 ¯0.714 ¯6.11 │2.34 1.2 1.71 2.51│││2│││3 4│3 4││
││¯0.382 0.481 2.37 ¯3.96 │0.916 0.76 0.0855 1.19││└─┘│└───┴───┘│
││¯7.04 3.82 2.11 ¯0.733│0.867 0.535 ¯1.29 1.75││ │ │
│└───────────────────────────┴────────────────────────┘│ │ │
└──────────────────────────────────────────────────────┴───┴─────────┘

Everything looks fine. To make sure our function is doing the right thing, let's try it with arrays of other ranks:

In [13]:
T ← {⍺←3 ⋄ m b←(⊂⍵)RN¨(⍺ ⍺) ⍺ ⋄ ('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m) ⋄ Assert b≡m+.×x}  ⍝ test
T ⍬ ⋄ T 2 ⋄ T 2 2 ⋄ T 2 2 2  ⍝ scalar, vector, matrix and 3D array

┌──────────────────┬───┬────┐
│x │⍴x │⍴¨x │
├──────────────────┼───┼────┤
│┌─────┬────┬─────┐│┌─┐│┌┬┬┐│
││0.878│1.29│¯1.46│││3│││││││
│└─────┴────┴─────┘│└─┘│└┴┴┘│
└──────────────────┴───┴────┘
┌─────────────────────────────────────┬───┬───────┐
│x │⍴x │⍴¨x │
├─────────────────────────────────────┼───┼───────┤
│┌─────────┬───────────┬─────────────┐│┌─┐│┌─┬─┬─┐│
││0.605 ¯20│¯0.416 3.03│¯0.00192 15.1│││3│││2│2│2││
│└─────────┴───────────┴─────────────┘│└─┘│└─┴─┴─┘│
└─────────────────────────────────────┴───┴───────┘
┌─────────────────────────────────────────┬───┬─────────────┐
│x │⍴x │⍴¨x │
├─────────────────────────────────────────┼───┼─────────────┤
│┌───────────┬──────────────┬────────────┐│┌─┐│┌───┬───┬───┐│
││0.515 0.283│ 0.811 ¯0.0183│¯0.911 0.113│││3│││2 2│2 2│2 2││
││0.904 1.11 │¯1.22 ¯0.0321│ 1.8 2.12 ││└─┘│└───┴───┴───┘│
│└───────────┴──────────────┴────────────┘│ │ │
└─────────────────────────────────────────┴───┴─────────────┘
┌─────────────────────────────────────────────────┬───┬───────────────────┐
│x │⍴x │⍴¨x │
├─────────────────────────────────────────────────┼───┼───────────────────┤
│┌──────────────┬────────────────┬───────────────┐│┌─┐│┌─────┬─────┬─────┐│
││ ¯9.47 1.01│ 26.6 1.08│¯10.8 ¯0.374│││3│││2 2 2│2 2 2│2 2 2││
││ 0.262 4.85│ ¯0.000929 10.8 │ 0.564 ¯10 ││└─┘│└─────┴─────┴─────┘│
││ │ │ ││ │ │
││ 12.2 ¯11.8 │¯16.3 14 │ 4.31 ¯2.69 ││ │ │
││¯10.1 ¯22.9 │ 2.34 2.21│ 10.2 13.8 ││ │ │
│└──────────────┴────────────────┴───────────────┘│ │ │
└─────────────────────────────────────────────────┴───┴───────────────────┘

# Each nested

Applying a function like this, to each of the nested elements of the array but keeping the outer structure, looks like a useful operation. So, we are going to define an operator.

The `ND` function that we defined before could easily be modified to be an *each nested* operator that works with any function:

In [14]:
_EN ← {⊂⍉⍤¯1⍉⍺ ⍺⍺⍤1 2⍥(↑[0])⍵}   ⍝ each nested
 ND ← ⌹_EN
_ ← T¨⍬ 2 (2 2) (2 2 2)

However, this operator is not very general. It will work when we have matrices as right argument and vectors on the left, but it will not work for arrays of any rank. For example, let's try to use the matrix divide primitive (`⌹`) to perform a least-squares fitting with two vectors:

In [15]:
a b ← ?3 3⍴¨0                ⍝ random input data
'ab⍴⌹' Table a b (⍴a) (a⌹b)  ⍝ flat arrays just work
a b ← (⊂4 2)RN¨3 3           ⍝ random nested arrays
'ab⍴¨' Table a b (⍴a) (⍴¨a)
a ⌹_EN b

┌────────────────────┬───────────────────┬───┬──────┐
│a │b │⍴ │⌹ │
├────────────────────┼───────────────────┼───┼──────┤
│┌─────┬──────┬─────┐│┌─────┬─────┬─────┐│┌─┐│┌────┐│
││0.278│0.0941│0.891│││0.192│0.719│0.966│││3│││0.66││
│└─────┴──────┴─────┘│└─────┴─────┴─────┘│└─┘│└────┘│
└────────────────────┴───────────────────┴───┴──────┘

┌───────────────────────────────────┬───────────────────────────────────┬───┬─────────────┐
│a │b │⍴ │¨ │
├───────────────────────────────────┼───────────────────────────────────┼───┼─────────────┤
│┌──────────┬───────────┬──────────┐│┌───────────┬───────────┬─────────┐│┌─┐│┌───┬───┬───┐│
││3.17 2.57 │2.12 4.22 │2.11 2.37 │││4.74 4.89 │0.575 3.59 │2.49 1.83│││3│││4 2│4 2│4 2││
││2.61 0.258│1.18 2.52 │2.39 0.931│││1.08 0.275│1.14 4.55 │4.2 4.9 ││└─┘│└───┴───┴───┘│
││1.91 4.93 │0.619 3.04 │2.25 4.58 │││0.797 3.94 │3.52 0.513│2.94 2.53││ │ │
││1.07 1.49 │3.01 0.173│4.28 2.53 │││5 2.38 │4.56 2.4 │4.89 1.03││ │ │
│└──────────┴───────────┴──────────┘│└───────────┴───────────┴─────────┘│ │ │
└───────────────────────────────────┴───────────────────────────────────┴───┴─────────────┘

RANK ERROR
_EN[0] _EN←{⊂⍉⍤¯1⍉⍺ ⍺⍺⍤1 2⍥(↑[0])⍵}   ⍝ each nested
                       ∧


Instead of hardcoding the ranks inside the operator, we can check what is the rank of our nested elements looking at the first element in `⍵`. We will use this rank to transpose subarrays (`⍉⍤(⍴⍴⊃⍵)`). The rank to apply the given function is calculated from the ranks of the left and right arguments (`⊃∘⍴∘⍴¨⍺ ⍵`):

In [16]:
'⍴a' '⍴b' '⍴⍴⊃b' '⊃∘⍴∘⍴¨a b' Table (⍴a) (⍴b) (⍴⍴⊃b) (⊃∘⍴∘⍴¨a b)
_EN ← {⍺←⊢ ⋄ ⊂∘⍉⍤(⍴⍴⊃⍵)∘⍉ ⍺ ⍺⍺⍤(⊃∘⍴∘⍴¨⍺ ⍵)⍥(↑[0]) ⍵}   ⍝ each nested
 MD ← ⌹_EN
_ ← T¨⍬ 2 (2 2) (2 2 2)
'ab⌹' Table a b (a ⌹_EN b)

┌───┬───┬────┬─────────┐
│⍴a │⍴b │⍴⍴⊃b│⊃∘⍴∘⍴¨a b│
├───┼───┼────┼─────────┤
│┌─┐│┌─┐│┌─┐ │┌─┬─┐ │
││3│││3│││2│ ││1│1│ │
│└─┘│└─┘│└─┘ │└─┴─┘ │
└───┴───┴────┴─────────┘

┌───────────────────────────────────┬───────────────────────────────────┬─────────────┐
│a │b │⌹ │
├───────────────────────────────────┼───────────────────────────────────┼─────────────┤
│┌──────────┬───────────┬──────────┐│┌───────────┬───────────┬─────────┐│┌───────────┐│
││3.17 2.57 │2.12 4.22 │2.11 2.37 │││4.74 4.89 │0.575 3.59 │2.49 1.83│││0.742 0.797││
││2.61 0.258│1.18 2.52 │2.39 0.931│││1.08 0.275│1.14 4.55 │4.2 4.9 │││0.708 0.36 ││
││1.91 4.93 │0.619 3.04 │2.25 4.58 │││0.797 3.94 │3.52 0.513│2.94 2.53│││0.477 1.47 ││
││1.07 1.49 │3.01 0.173│4.28 2.53 │││5 2.38 │4.56 2.4 │4.89 1.03│││0.574 0.526││
│└──────────┴───────────┴──────────┘│└───────────┴───────────┴─────────┘│└───────────┘│
└───────────────────────────────────┴───────────────────────────────────┴─────────────┘

The `_EN` operator can now be used either to solve a linear system or perform a least-squares fitting, and it will do the right thing.

Of course, `⌹` is not the only function we can use with `_EN`. As a last example, let's take the `LF` function from the [Linear Fitting notebook](https://github.com/Dyalog/dyalog-jupyter-notebooks/blob/master/Linear%20Fitting.ipynb), which performs a linear fitting given a list of two vectors and returns the intercept, the slope and the R-squared value. For convenience, we also define a dyadic linear fitting function:

In [17]:
_R2 ← {1-(+/×⍨⍺-⍺⍺⍵)÷+/×⍨(⊢-+/÷≢)⍺}                                            ⍝ R-squared value
LF  ← {F←{((a+b∘×)_R2/⍺),⍨a b←⍵} ⋄ 0=⎕NC'⍺': ⍵F⊃⌹∘(1,⍪⍤2⍤⊢)/⍵ ⋄ ⍵F⍺(⌹/⍵-⍺ 0)}  ⍝ linear fitting
DLF ← LF⍤,⍥⊂                                                                   ⍝ dyadic linear fitting
⍝ eg with non nested values
'xy' Table x y←5×?0⍴⍨¨4 4
'abr' Table y DLF x

┌───────────────────┬─────────────────────┐
│x │y │
├───────────────────┼─────────────────────┤
│┌────┬───┬────┬───┐│┌─────┬────┬───┬────┐│
││4.81│4.6│2.99│1.9│││0.158│3.78│4.8│4.43││
│└────┴───┴────┴───┘│└─────┴────┴───┴────┘│
└───────────────────┴─────────────────────┘

┌──────┬───────┬───────┐
│a │b │r │
├──────┼───────┼───────┤
│┌────┐│┌─────┐│┌─────┐│
││7.12│││¯1.07│││0.483││
│└────┘│└─────┘│└─────┘│
└──────┴───────┴───────┘

If we have nested data, we use the `_EN` operator:

In [18]:
'xy' Table x y←(⊂3 2)RN¨4 4
'abr' Table y DLF _EN x

┌───────────────────────────────────────────┬─────────────────────────────────────────┐
│x │y │
├───────────────────────────────────────────┼─────────────────────────────────────────┤
│┌─────────┬──────────┬──────────┬─────────┐│┌─────────┬─────────┬─────────┬─────────┐│
││3.55 4.55│0.972 2.91│4.89 0.969│4.4 2.03│││2.78 1.34│3.54 2.88│1.63 1.96│4.44 2.1 ││
││4.51 3.44│0.692 4.13│4.14 3.74 │3.21 2.39│││1.63 3.23│4.56 1.87│1.01 2.25│3.88 1.53││
││4.94 4.67│2.06 4.39│1.71 1.81 │3.12 1.65│││3.5 3.88│4.35 3.76│2.91 2.1 │1.37 2.46││
│└─────────┴──────────┴──────────┴─────────┘│└─────────┴─────────┴─────────┴─────────┘│
└───────────────────────────────────────────┴─────────────────────────────────────────┘

┌───────────┬────────────────┬───────────────┐
│a │b │r │
├───────────┼────────────────┼───────────────┤
│┌────┬────┐│┌───────┬──────┐│┌───────┬─────┐│
││3.84│2.45│││¯0.216 │¯0.147│││0.1 │0.124││
│├────┼────┤│├───────┼──────┤│├───────┼─────┤│
││5.47│1.21│││¯0.862 │0.296 │││0.743 │0.091││
│├────┼────┤│├───────┼──────┤│├───────┼─────┤│
││3.21│1.35│││¯0.0582│0.544 │││0.00453│0.958││
│└────┴────┘│└───────┴──────┘│└───────┴─────┘│
└───────────┴────────────────┴───────────────┘

It is also possible to use the `_EN` operator when both arguments do not have the same rank.

In [19]:
'xy' Table x (y1←5×?4⍴0)
'abr' Table y1 DLF _EN x

┌───────────────────────────────────────────┬──────────────────────┐
│x │y │
├───────────────────────────────────────────┼──────────────────────┤
│┌─────────┬──────────┬──────────┬─────────┐│┌────┬─────┬────┬────┐│
││3.55 4.55│0.972 2.91│4.89 0.969│4.4 2.03│││3.98│0.542│1.82│1.94││
││4.51 3.44│0.692 4.13│4.14 3.74 │3.21 2.39││└────┴─────┴────┴────┘│
││4.94 4.67│2.06 4.39│1.71 1.81 │3.12 1.65││ │
│└─────────┴──────────┴──────────┴─────────┘│ │
└───────────────────────────────────────────┴──────────────────────┘

┌──────────────┬──────────────┬──────────────┐
│a │b │r │
├──────────────┼──────────────┼──────────────┤
│┌──────┬─────┐│┌─────┬──────┐│┌─────┬──────┐│
││0.748 │0.656│││0.383│0.541 │││0.221│0.333 ││
│├──────┼─────┤│├─────┼──────┤│├─────┼──────┤│
││¯0.088│4.1 │││0.689│¯0.592│││0.693│0.0967││
│├──────┼─────┤│├─────┼──────┤│├─────┼──────┤│
││¯0.477│1.46 │││0.862│0.195 │││0.773│0.0494││
│└──────┴─────┘│└─────┴──────┘│└─────┴──────┘│
└──────────────┴──────────────┴──────────────┘

But take into account that `_EN` will deduce the shape of the elements from the first element in the right argument, so you may have to switch the arguments:

In [20]:
'xy' Table (x1←5×?4⍴0) y
'abr' Table x1 DLF⍨ _EN y

┌─────────────────────┬─────────────────────────────────────────┐
│x │y │
├─────────────────────┼─────────────────────────────────────────┤
│┌────┬─────┬───┬────┐│┌─────────┬─────────┬─────────┬─────────┐│
││4.31│0.968│3.4│3.46│││2.78 1.34│3.54 2.88│1.63 1.96│4.44 2.1 ││
│└────┴─────┴───┴────┘││1.63 3.23│4.56 1.87│1.01 2.25│3.88 1.53││
│ ││3.5 3.88│4.35 3.76│2.91 2.1 │1.37 2.46││
│ │└─────────┴─────────┴─────────┴─────────┘│
└─────────────────────┴─────────────────────────────────────────┘

┌───────────┬───────────────┬───────────────┐
│a │b │r │
├───────────┼───────────────┼───────────────┤
│┌────┬────┐│┌──────┬──────┐│┌──────┬──────┐│
││3.73│3.35│││¯0.209│¯0.424│││0.0636│0.929 ││
│├────┼────┤│├──────┼──────┤│├──────┼──────┤│
││5.38│1.38│││¯0.861│0.277 │││0.518 │0.296 ││
│├────┼────┤│├──────┼──────┤│├──────┼──────┤│
││4.44│3.54│││¯0.462│¯0.162│││0.28 │0.0665││
│└────┴────┘│└──────┴──────┘│└──────┴──────┘│
└───────────┴───────────────┴───────────────┘

# Summary

The `_EN` operator will apply the given function to each of the nested elements of its arguments, taking as model the first element of the right argument, while keeping the outer structure.

In [21]:
]defs _EN

_EN ← {⍺←⊢ ⋄ ⊂∘⍉⍤(⍴⍴⊃⍵)∘⍉⍺ ⍺⍺⍤(⊃∘⍴∘⍴¨⍺ ⍵)⍥(↑[0])⍵}